In [1]:
# https://www.youtube.com/watch?v=3QiPPX-KeSc

import socket
import threading
import random
import string
import time
import pickle

HEADER = 64
PORT = 8080
SERVER = "0.0.0.0"
ADDR = (SERVER, PORT)
FORMAT = 'utf-8'
DISCONNECT_MESSAGE = ['!DISCONNECT']

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(ADDR)

connections = []
rooms = []
stack = [[0,0], [1,0], [1,0], [2,0], [2,0], [3,0], [3,0], [4,0], [4,0], [5,0], [5,0], [6,0], [6,0], [7,0], [7,0], [8,0], [8,0], [9,0], [9,0], [10,0], [10,0], [11,0], [11,0], [12,0], [12,0], 
             [0,1], [1,1], [1,1], [2,1], [2,1], [3,1], [3,1], [4,1], [4,1], [5,1], [5,1], [6,1], [6,1], [7,1], [7,1], [8,1], [8,1], [9,1], [9,1], [10,1], [10,1], [11,1], [11,1], [12,1], [12,1], 
             [0,2], [1,2], [1,2], [2,2], [2,2], [3,2], [3,2], [4,2], [4,2], [5,2], [5,2], [6,2], [6,2], [7,2], [7,2], [8,2], [8,2], [9,2], [9,2], [10,2], [10,2], [11,2], [11,2], [12,2], [12,2], 
             [0,3], [1,3], [1,3], [2,3], [2,3], [3,3], [3,3], [4,3], [4,3], [5,3], [5,3], [6,3], [6,3], [7,3], [7,3], [8,3], [8,3], [9,3], [9,3], [10,3], [10,3], [11,3], [11,3], [12,3], [12,3], 
             [0,4], [0,4], [0,4], [0,4], [5,4], [5,4], [5,4], [5,4]]
stackByRoom = []
heapByRoom = []

In [2]:
def handle_client(conn, addr):
    global connections
    global rooms
    global stackByRoom
    global heapByRoom
    global stack
    connections.append(conn)
    print(f"[INFO] New connection, {addr} connected.")
    connected = True
    while connected:
        msg_lenght = conn.recv(HEADER).decode(FORMAT)
        if msg_lenght:
            msg_lenght = int(msg_lenght)
            msg = pickle.loads(conn.recv(msg_lenght))
            if msg == DISCONNECT_MESSAGE:
                connected = False
                print(f"[INFO] A client disconnected")
                print(f"[ACTIVE CONNECTION] {threading.activeCount() - 9 }")
                currentRoom = ""
                currentUsername = ""
                for r in rooms:
                    if r[0] == conn:
                        currentRoom = r[1]
                        currentUsername = r[2]
                for r in rooms:
                    if r[1] == currentRoom:
                        if r[0] == conn:
                            print(f"[{addr}] {currentUsername} has been diconnected from room {currentRoom}.")
                            rooms.remove(r)
                counter = 0
                for r in rooms:
                    if r[1] == currentRoom:
                        counter = counter + 1
                if counter > 0:
                    print(f"[INFO] Sending to players in room {currentRoom} updated player list.")
                    messageToSend = ["setPlayers"]
                    for r in rooms:
                        if r[1] == currentRoom:
                            messageToSend.append(r[2])
                    for r in rooms:
                        if r[1] == currentRoom and r[0] != conn:
                            r[0].send(pickle.dumps(messageToSend))
                elif counter == 0:
                    for s in stackByRoom:
                        if s[0] == currentRoom:
                            stackByRoom.remove(s)
                    for h in heapByRoom:
                        if h[0] == currentRoom:
                            heapByRoom.remove(h)
                            print(f"[{addr}] {currentUsername} was the last player in room {currentRoom}, stack and heap for this room has been deleted.")
            elif msg[1] == "createRoom":
                randomRoomNum = random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits) + random.choice(string.ascii_uppercase + string.digits)
                messageToSend = ['joinNow',f'{randomRoomNum}']
                rooms.append([conn, randomRoomNum, msg[0]])
                print(f"[{addr}] {msg[0]} created room number {randomRoomNum}")
                conn.send(pickle.dumps(messageToSend))
            elif msg[1] == "joinRoom":
                counter = 0
                for r in rooms:
                    if r[1] == msg[2]:
                        counter = counter + 1
                        if r[0] == conn:
                            print(f"[{addr}] {msg[0]} was already in room number {msg[2]}. He has been diconnected.")
                            rooms.remove(r)
                if counter != 0 and counter < 4:
                    counterCheckUsername = 0
                    for r in rooms:
                        if r[1] == msg[2]:
                            if r[2] == msg[0]:
                                counterCheckUsername = counterCheckUsername + 1
                    if (counterCheckUsername == 0):
                        rooms.append([conn, msg[2], msg[0]])
                        print(f"[{addr}] {msg[0]} joined room number {msg[2]}. Sending update...")
                        messageToSend = ['joinNow',f'{msg[2]}']
                        conn.send(pickle.dumps(messageToSend))
                        messageToSend = ["setPlayers"]
                        for r in rooms:
                            if r[1] == msg[2]:
                                messageToSend.append(r[2])
                        for r in rooms:
                            if r[1] == msg[2] and r[0] != conn:
                                r[0].send(pickle.dumps(messageToSend))
                    else:
                        messageToSend = ['error','usernameNotValid']
                        conn.send(pickle.dumps(messageToSend))
                        print(f"[{addr}] {msg[0]} tried to join room number {msg[2]}, but username was already used.")
                else:
                    messageToSend = ['error','roomCodeNotValid']
                    conn.send(pickle.dumps(messageToSend))
                    print(f"[{addr}] {msg[0]} tried to join room number {msg[2]}, but room doesn't exist or is full.")
            elif msg[1] == "getPlayers":
                print(f"[INFO] {msg[0]} want players in room {msg[2]}.")
                messageToSend = ['setPlayers']
                for r in rooms:
                    if r[1] == msg[2]:
                        messageToSend.append(r[2])
                conn.send(pickle.dumps(messageToSend))
            elif msg[1] == "quitRoom":
                for r in rooms:
                    if r[1] == msg[2]:
                        if r[0] == conn:
                            print(f"[{addr}] {msg[0]} has been diconnected from room {msg[2]}.")
                            rooms.remove(r)
                counter = 0
                for r in rooms:
                    if r[1] == msg[2]:
                        counter = counter + 1
                if counter > 0:
                    print(f"[INFO] Sending to players in room {msg[2]} updated player list.")
                    messageToSend = ['setPlayers']
                    for r in rooms:
                        if r[1] == msg[2]:
                            messageToSend.append(r[2])
                    for r in rooms:
                        if r[1] == msg[2] and r[0] != conn:
                            r[0].send(pickle.dumps(messageToSend))
            elif msg[1] == "startGame":
                counter = 0
                for r in rooms:
                    if r[1] == msg[2]:
                        counter = counter + 1
                if counter < 0:
                    messageToSend = ['error','notAValidNumberOfPerson']
                    print(f"[INFO] {msg[0]} wanted to start room {msg[2]}, but there wasn't enough players.")
                    conn.send(pickle.dumps(messageToSend))
                else:
                    tempStack = stack[:]
                    random.shuffle(tempStack)
                    stackByRoom.append([msg[2],tempStack])
                    for s in stackByRoom:
                        if s[0] == msg[2]:
                            roomPlaceInStack = stackByRoom.index(s)
                    counter = 0
                    hands = []
                    for r in rooms:
                        if r[1] == msg[2]:
                            counter = counter + 1
                    for c in range(0, counter):
                        tmpList = []
                        for i in range(0, 7):
                            tmpList.append(stackByRoom[roomPlaceInStack][1].pop(0))
                        hands.append(tmpList)
                    while ((stackByRoom[roomPlaceInStack][1][0][1] == 4) | (stackByRoom[roomPlaceInStack][1][0][0] in range(10, 13))) : 
                        random.shuffle(stackByRoom[roomPlaceInStack][1])
                    heapCard = stackByRoom[roomPlaceInStack][1].pop(0)
                    heapByRoom.append([msg[2],heapCard])
                    print(f"[INFO] {msg[0]} started room {msg[2]}.")
                    counter = 0
                    for r in rooms:
                        if r[1] == msg[2]:
                            messageToSend = ['startNow']
                            messageToSend.append(heapCard)
                            messageToSend.append(hands[counter])
                            if counter == 0:
                                messageToSend.append('yourTurn')
                            r[0].send(pickle.dumps(messageToSend))
                            counter = counter + 1
            elif msg[1] == "placing":
                print(f"[INFO] {msg[0]} placed card {msg[3]} in room {msg[2]}.")
                playersPlaying = []
                counter = -1
                indexCurrentPlayer = 0
                currentPlayerUsername = ""
                for r in rooms:
                    if r[1] == msg[2]:
                        counter = counter + 1
                        if r[2] == msg[0]:
                            indexCurrentPlayer = counter
                        playersPlaying.append(r)
                for player in playersPlaying:
                    messageToSend = ['updateHeap']
                    messageToSend.append(msg[3])
                    messageToSend.append(playersPlaying[indexCurrentPlayer][2])
                    messageToSend.append(msg[4])
                    for heap in heapByRoom:
                        if heap[0] == msg[2]:
                            heap[1] = msg[3]
                    if indexCurrentPlayer+1 == len(playersPlaying):
                        if playersPlaying.index(player) == 0:
                            messageToSend.append('yourTurn')
                    elif playersPlaying.index(player) == indexCurrentPlayer+1:
                        messageToSend.append('yourTurn')
                    player[0].send(pickle.dumps(messageToSend))
            elif msg[1] == "demandingCard":
                print(f"[INFO] {msg[0]} ask for a card in room {msg[2]}.")
                playersPlaying = []
                counter = -1
                indexCurrentPlayer = 0
                for r in rooms:
                    if r[1] == msg[2]:
                        counter = counter + 1
                        if r[2] == msg[0]:
                            indexCurrentPlayer = counter
                        playersPlaying.append(r)
                for player in playersPlaying:
                    messageToSend = ['askedCard']
                    messageToSend.append(playersPlaying[indexCurrentPlayer][2])
                    messageToSend.append(msg[3])
                    if indexCurrentPlayer+1 == len(playersPlaying):
                        if playersPlaying.index(player) == 0:
                            messageToSend.append('yourTurn')
                    elif playersPlaying.index(player) == indexCurrentPlayer+1:
                        messageToSend.append('yourTurn')
                    if player[0] != conn:
                        player[0].send(pickle.dumps(messageToSend))
                    else:
                        messageToSend = ['newCard']
                        messageToSend.append(playersPlaying[indexCurrentPlayer][2])
                        messageToSend.append(msg[3])
                        for s in stackByRoom:
                            if s[0] == msg[2]:
                                roomPlaceInStack = stackByRoom.index(s)
                        messageToSend.append(stackByRoom[roomPlaceInStack][1].pop(0))
                        conn.send(pickle.dumps(messageToSend))
            elif msg[1] == "winning":
                print(f"[INFO] {msg[0]} won the room {msg[2]}.")
                for s in stackByRoom:
                    if s[0] == msg[2]:
                        stackByRoom.remove(s)
                for h in heapByRoom:
                    if h[0] == msg[2]:
                        heapByRoom.remove(h)
                roomToDelete = []
                for r in rooms:
                    if r[1] == msg[2]:
                        roomToDelete.append(r)
                        messageToSend = ['partyEnd']
                        messageToSend.append(msg[0])
                        r[0].send(pickle.dumps(messageToSend))
                for r in roomToDelete:
                    rooms.remove(r)
                print(f"[{addr}] Deleting room {msg[2]}.")
                #conn.send(msg.encode(FORMAT))
                # Pour envoyer a tous les clients
                #for connClient in connections:
                    #connClient.send(msg.encode(FORMAT))
    conn.close()
    connections.remove(conn)

In [3]:
def start():
    server.listen()
    print(f"[LISTENING] Server is listening on {SERVER}")
    while True:
        conn, addr = server.accept()
        thread = threading.Thread(target=handle_client, args=(conn, addr))
        thread.start()
        print(f"[ACTIVE CONNECTION] {threading.activeCount() - 8 }")
print("[STARTING] le serveur démarre")

[STARTING] le serveur démarre


In [ ]:
start()

[LISTENING] Server is listening on 0.0.0.0
[INFO] New connection, ('127.0.0.1', 51428) connected.[ACTIVE CONNECTION] -1

[('127.0.0.1', 51428)] Eddy created room number T1IEP
[INFO] Eddy want players in room T1IEP.
[INFO] Eddy started room T1IEP.
[INFO] Eddy placed card [11, 3] in room T1IEP.
[INFO] Eddy placed card [9, 3] in room T1IEP.
[INFO] Eddy placed card [9, 1] in room T1IEP.
[INFO] Eddy placed card [9, 1] in room T1IEP.
[INFO] Eddy ask for a card in room T1IEP.
[INFO] A client disconnected
[ACTIVE CONNECTION] -2
[('127.0.0.1', 51428)] Eddy has been diconnected from room T1IEP.
[('127.0.0.1', 51428)] Eddy was the last player in room T1IEP, stack and heap for this room has been deleted.
[INFO] New connection, ('127.0.0.1', 51437) connected.[ACTIVE CONNECTION] -1

[INFO] New connection, ('127.0.0.1', 51567) connected.[ACTIVE CONNECTION] 0

[('127.0.0.1', 51567)] eddy created room number M9SIN
[INFO] eddy want players in room M9SIN.
[('127.0.0.1', 51437)] eddy2 joined room number M